In [7]:
import requests
import json
env = open('../.env').readlines()
DISCORD_APPLICATION_ID = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith('DEV_DISCORD_APPLICATION_ID')][0]
DISCORD_BOT_TOKEN = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith('DEV_DISCORD_BOT_TOKEN')][0]

In [14]:
# CREATE COMMAND
# https://discord.com/developers/docs/interactions/application-commands#making-a-global-command
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"

# This is an example CHAT_INPUT or Slash Command, with a type of 1
json = {
    "name": "starter",
    "description": "Generate a conversation starter given some topics",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','",
            "type": 3,
            "required": False,
            # "choices": [
            #     {
            #         "name": "Common",
            #         "value": "ice breaker,travel"
            #     },
            #     {
            #         "name": "Scientific",
            #         "value": "philosophy,science,physic,mathematic,biology"
            #     },
            # ]
        },
    ]
}

# For authorization, you can use either your bot token
headers = {
    "Authorization": f"Bot {DISCORD_BOT_TOKEN}"
}

r = requests.post(url, headers=headers, json=json)
r.text

'{"id": "920656302420684820", "application_id": "920612693864493066", "version": "920656302420684821", "default_permission": true, "default_member_permissions": null, "type": 1, "name": "starter", "name_localizations": null, "description": "Generate a conversation starter given some topics", "description_localizations": null, "dm_permission": null, "options": [{"type": 3, "name": "topics", "name_localizations": null, "description": "List of topics separated by \',\'", "description_localizations": null}]}'

In [12]:
# LIST COMMAND

url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"


# For authorization, you can use either your bot token
headers = {
    "Authorization": f"Bot {DISCORD_BOT_TOKEN}"
}


r = requests.get(url, headers=headers)
r.json()

[{'id': '920655953999835167',
  'application_id': '920612693864493066',
  'version': '920655953999835168',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'starter',
  'description': 'Generate a conversation starter given some topics',
  'dm_permission': None,
  'options': [{'type': 3,
    'name': 'topics',
    'description': "List of topics separated by ','"}]}]

In [13]:
# COMMAND DELETE

url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/920655953999835167"


# For authorization, you can use either your bot token
headers = {
    "Authorization": f"Bot {DISCORD_BOT_TOKEN}"
}


r = requests.delete(url, headers=headers)
r.text

''